# 05. LASSO Model

**Mô hình:** LASSO Regression (L1 regularization)

**Thư viện:** sklearn.linear_model.Lasso

**Metrics:** RMSFE, MAPE

In [ ]:
# Import libraries

In [ ]:
# Load processed data

In [ ]:
# LASSO model with GridSearchCV

In [ ]:
# Evaluation

In [ ]:
# Save results